In [16]:
import pandas as pd
import numpy as np
import jqdatasdk as jq
import statsmodels.api as sm
import time,datetime
import os

In [2]:
jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [ ]:
# 先分组，5因子分组：市值2组，其余3组

In [7]:
# 生成分组标识
bin_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
for col in ['B_M', 'OP','pe_ratio']:
    if isinstance(bin_df[col][0],float):
        bin_df[col] = bin_df[col].apply(lambda x : np.nan if x<0 else x)
bin_df = bin_df.dropna(how='any',axis=0)

## 分组组数
bin_num = 3

for i in range(8):
    
    year = str(2010+i)+'-12-31'
    temp_df_0 = bin_df[bin_df.date==year]
    
    # 计算当年总行数
    stock_num = len(temp_df_0)
    
    for bin_kind in ['B_M','OP','Inv']:
        
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst
            
    for bin_kind in ['Size']:
        
        # Size只分两组
        bin_num_size = 2
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num_size):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num_size)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst    
    
    temp_df_1 = bin_df[bin_df.date==year]
    temp_df_1 = temp_df_1.sort_values('Size') # 和df_0对齐，最后一次是按照Size排序的
    temp_df_0['S']=temp_df_1['Size']
    temp_df_0.to_csv('E:/Stock_Data/factor_portfolio/bin_data'+year+'.csv',index=True)

In [8]:
# 按要求生成投资组合

# 保存在字典：除Size以外的另一个组别 -> 年份 -> bin2 -> 组合，stock code list

portfolio={}

for bin2 in ['B_M','OP','Inv','Size']:
    portfolio[bin2] = {}
    
    for i in range(8):
        year = str(2010+i)+'-12-31'
        portfolio[bin2][str(2011+i)]={}
        
        temp_df_0 = pd.read_csv('E:/Stock_Data/factor_portfolio/bin_data'+year+'.csv',index_col=0)
        bin2_lst = list(set(temp_df_0[bin2]))
        
        for bin2_ in bin2_lst:
            temp_df_1 = temp_df_0[temp_df_0[bin2]==bin2_]
            list_code_value = [list(temp_df_1[['code','S']].iloc[i,:]) for i in range(len(temp_df_1))]
            portfolio[bin2][str(2011+i)][bin2_] = list_code_value
            
# 保存字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','w')
f.write(str(portfolio))
f.close()

In [9]:
# 读取字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()

In [18]:
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

In [19]:
stock_download_return

['000001.XSHE',
 '000002.XSHE',
 '000004.XSHE',
 '000005.XSHE',
 '000006.XSHE',
 '000008.XSHE',
 '000009.XSHE',
 '000011.XSHE',
 '000012.XSHE',
 '000014.XSHE',
 '000016.XSHE',
 '000017.XSHE',
 '000018.XSHE',
 '000019.XSHE',
 '000020.XSHE',
 '000021.XSHE',
 '000022.XSHE',
 '000023.XSHE',
 '000024.XSHE',
 '000025.XSHE',
 '000026.XSHE',
 '000027.XSHE',
 '000028.XSHE',
 '000030.XSHE',
 '000031.XSHE',
 '000032.XSHE',
 '000034.XSHE',
 '000036.XSHE',
 '000037.XSHE',
 '000039.XSHE',
 '000040.XSHE',
 '000042.XSHE',
 '000043.XSHE',
 '000045.XSHE',
 '000046.XSHE',
 '000048.XSHE',
 '000049.XSHE',
 '000050.XSHE',
 '000055.XSHE',
 '000056.XSHE',
 '000058.XSHE',
 '000059.XSHE',
 '000060.XSHE',
 '000061.XSHE',
 '000062.XSHE',
 '000063.XSHE',
 '000065.XSHE',
 '000066.XSHE',
 '000069.XSHE',
 '000070.XSHE',
 '000078.XSHE',
 '000088.XSHE',
 '000089.XSHE',
 '000090.XSHE',
 '000096.XSHE',
 '000099.XSHE',
 '000100.XSHE',
 '000150.XSHE',
 '000151.XSHE',
 '000153.XSHE',
 '000157.XSHE',
 '000158.XSHE',
 '000159

In [20]:
# 对于某一投资组合 bin2 计算所有收益率序列
address = 'E:/Stock_Data/stock_return_data/'
bin2s = ['B_M','OP','Inv','Size']
return_inf_used_stock = list(pd.read_csv('E:/Stock_Data/return_inf_used_stock.csv').iloc[:,0])
stock_download_return_temp = os.listdir('E:/Stock_Data/stock_return_data/')
stock_download_return = [x[:-4] for x in stock_download_return_temp]

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r*weight
                else:
                    r_sigma = r_sigma+r*weight
                
                tol_weight += weight       
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/tol_weight
            r_mean.columns = [bin1_bin2_type]
            if cnt_bin == 0:
                r_save = r_mean
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/'+bin2+'.csv',index=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [ ]:
# 因子计算

In [21]:
# SMB small minus big - Size
Size = pd.read_csv('E:/Stock_Data/factor_portfolio/Size.csv',index_col=0).iloc[:1943,:]
SMB = Size['Size1'] - Size['Size2']
SMB = pd.DataFrame(SMB,columns=['SMB'])

In [22]:
# HML high minus low - B_M
B_M = pd.read_csv('E:/Stock_Data/factor_portfolio/B_M.csv',index_col=0).iloc[:1943,:]
HML = B_M['B_M3'] - B_M['B_M1']
HML = pd.DataFrame(HML,columns=['HML'])

In [23]:
# RMW robust minus weak - OP
OP = pd.read_csv('E:/Stock_Data/factor_portfolio/OP.csv',index_col=0).iloc[:1943,:]
RMW = OP['OP3'] - OP['OP1']
RMW.columns = ['RMW']
RMW = pd.DataFrame(RMW,columns=['RMW'])

In [24]:
# CMA conservative minus aggressive - Inv
Inv = pd.read_csv('E:/Stock_Data/factor_portfolio/Inv.csv',index_col=0).iloc[:1943,:]
CMA = Inv['Inv1'] - Inv['Inv3']
CMA = pd.DataFrame(CMA,columns=['CMA'])

In [26]:
# market risk，读取投资组合，选取一个标准的全部样本（全样本），计算加权均值，即 Rm

# 读取字典
f = open('E:/Stock_Data/factor_portfolio/portfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()
portfolio_0 = portfolio['Size']
## 第二层，不同年份
for i in range(8):
    year = str(2011+i)
    portfolio_1 = portfolio_0[year]
    tol_weight = 0 # 把总权重放置年份后，对每小组不再单独求和，即Size1+Size2=全股票
    cnt = 0 # 进对每一年清零计数

    ## 第三层，不同小组
    for bin1_bin2_type in portfolio_1.keys():


        portfolio_2 = portfolio_1[bin1_bin2_type]
        for code_weight in portfolio_2:

            code = code_weight[0]
            weight = code_weight[1]

            # 股票由于停牌时间过长被剔除
            if code not in stock_download_return:
                continue

            file_name = address+code+'.csv'
            r = pd.read_csv(file_name,index_col=0)

            # 读取到的序列过短，剔除
            if len(r)<1943:
                continue

            if cnt == 0:
                r_sigma = r*weight
            else:
                r_sigma = r_sigma+r*weight

            tol_weight += weight       
            cnt+=1  

    ### 求加权平均数，得到一年的收益率序列
    r_mean = r_sigma/tol_weight
    r_save = r_mean

    ### 删选当年的数据
    r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
    r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
    ### 每一年拼接
    if i == 0:
        r_save_year = r_save_1
    else:
        r_save_year = pd.concat([r_save_year,r_save_1],axis=0)

### 完成一个大组的每一年计算后，保存
r_save_year.columns = ['return']
r_save_year.to_csv('E:/Stock_Data/factor_portfolio/MarketRisk.csv',index=True)

MKT = r_save_year.iloc[:1943,:]

In [27]:
# rf
import re
rf_temp_0 = pd.read_csv('E:/Stock_Data/t_bill_one_year.csv')[['日期','收盘']]
rf_temp_0.index = rf_temp_0['日期'].apply(lambda x: x[:4]+'-'+re.findall('(.*?)年(.*?)月(.*?)日',x)[0][1].zfill(2)+'-'+re.findall('(.*?)年(.*?)月(.*?)日',x)[0][2].zfill(2))
rf_temp_1 = pd.merge(left = MKT,right = rf_temp_0,left_index = True,right_index=True,how='left')[['收盘']]
rf_temp_2 = rf_temp_1.fillna(rf_temp_1.mean())
rf_temp_2.columns = ['return']
rf = rf_temp_2/252/100
rf.to_csv('E:/Stock_Data/rf.csv')

In [29]:
# MKTF
MKTF = MKT - rf
MKTF.columns = ['MKTF']

In [ ]:
# 引入日历效应哑变量 - 星期
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['weekday']
trade_day_df['weekday'] = trade_day_df['weekday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").weekday()+1)

dummy_df = pd.get_dummies(trade_day_df['weekday'])
dummy_df = dummy_df[[1,2,3,4,5]]
dummy_df.index = sample_df.index

X = dummy_df.iloc[:1943,:]

In [30]:
# 检验各因子的周四效应

for factor in [MKTF,SMB,HML,CMA,RMW]:
    y = factor

    est = sm.OLS(y,X)
    result = est.fit()
    print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MKTF   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.992
Date:                Wed, 10 Apr 2019   Prob (F-statistic):             0.0932
Time:                        23:27:37   Log-Likelihood:                 5590.7
No. Observations:                1943   AIC:                        -1.117e+04
Df Residuals:                    1938   BIC:                        -1.114e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
1             -0.0005      0.001     -0.689      0.4

In [ ]:
# 依照回归结果分组，B_M以及OP单独分组

In [37]:
# 生成分组标识
bin_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
for col in ['B_M', 'OP','pe_ratio']:
    if isinstance(bin_df[col][0],float):
        bin_df[col] = bin_df[col].apply(lambda x : np.nan if x<0 else x)
bin_df = bin_df.dropna(how='any',axis=0)

## 分组组数
bin_num = 8

for i in range(8):
    
    year = str(2010+i)+'-12-31'
    temp_df_0 = bin_df[bin_df.date==year]
    
    # 计算当年总行数
    stock_num = len(temp_df_0)
    
    for bin_kind in ['B_M','OP']:
        
        # 生成分组标识
        bin_lst = []
        for i in range(bin_num):
            bin_lst += [bin_kind+str(i+1)]*int(stock_num/bin_num)
        if len(bin_lst) != stock_num:
            bin_lst += [bin_kind+str(i+1)]*(stock_num-len(bin_lst))
        
        temp_df_0 = temp_df_0.sort_values(bin_kind)
        temp_df_0[bin_kind] = bin_lst
    
    temp_df_1 = bin_df[bin_df.date==year]
    temp_df_1 = temp_df_1.sort_values('OP') # 和df_0对齐，最后一次是按照Size排序的
    temp_df_0['S']=temp_df_1['Size']
    temp_df_0.to_csv('E:/Stock_Data/factor_portfolio/B_M&OP'+year+'.csv',index=True)

In [38]:
# 按要求生成投资组合

# 保存在字典：除B_M以外的另一个组别 -> 年份 -> bin1 与 bin2 单独！ -> 组合，stock code list

portfolio={}

for bin2 in ['B_M','OP']:
    portfolio[bin2] = {}
    
    for i in range(8):
        year = str(2010+i)+'-12-31'
        portfolio[bin2][str(2011+i)]={}
        
        temp_df_0 = pd.read_csv('E:/Stock_Data/factor_portfolio/B_M&OP'+year+'.csv',index_col=0)
        bin2_lst = list(set(temp_df_0[bin2]))
        
        for bin2_ in bin2_lst:  
            temp_df_2 = temp_df_0[temp_df_0[bin2]==bin2_]
            list_code_value = [list(temp_df_2[['code','S']].iloc[i,:]) for i in range(len(temp_df_2))]
            portfolio[bin2][str(2011+i)][bin2_] = list_code_value
# 保存字典
f = open('E:/Stock_Data/factor_portfolio/B_M_OPportfolio.txt','w')
f.write(str(portfolio))
f.close()

In [39]:
# 读取字典
f = open('E:/Stock_Data/factor_portfolio/B_M_OPportfolio.txt','r')
a = f.read()
portfolio = eval(a)
f.close()

In [40]:
# 对于某一投资组合 bin2 计算所有收益率序列
address = 'E:/Stock_Data/stock_return_data/'

bin2s = ['B_M','OP']
return_inf_used_stock = list(pd.read_csv('E:/Stock_Data/return_inf_used_stock.csv').iloc[:,0])

## 第一层，分组依据
for bin2 in bin2s:
    portfolio_0 = portfolio[bin2]
    
    ## 第二层，不同年份
    for i in range(8):
        year = str(2011+i)
        portfolio_1 = portfolio_0[year]
        cnt_bin = 0
        
        ## 第三层，不同小组
        for bin1_bin2_type in portfolio_1.keys():
            cnt = 0
            tol_weight = 0
            portfolio_2 = portfolio_1[bin1_bin2_type]
            for code_weight in portfolio_2:
                
                code = code_weight[0]
                weight = code_weight[1]
                
                # 股票由于停牌时间过长被剔除
                if code not in stock_download_return:
                    continue
                
                file_name = address+code+'.csv'
                r = pd.read_csv(file_name,index_col=0)
                
                # 读取到的序列过短，剔除
                if len(r)<1943:
                    continue
                
                if cnt == 0:
                    r_sigma = r*weight
                else:
                    r_sigma = r_sigma+r*weight
                
                tol_weight += weight       
                cnt+=1  
            
            ### 求加权平均数，得到一小组的收益率序列
            r_mean = r_sigma/tol_weight
            r_mean.columns = [bin1_bin2_type]
            if cnt_bin == 0:
                r_save = r_mean
            else:
                r_save = pd.concat([r_save,r_mean],axis=1)
            cnt_bin += 1
        ### 删选当年的数据
        r_save_0 = r_save[r_save.index< str(int(year)+1)+'-01-01']
        r_save_1 = r_save_0[r_save_0.index> str(int(year)-1)+'-12-31']
        ### 每一年拼接
        if i == 0:
            r_save_year = r_save_1
        else:
            r_save_year = pd.concat([r_save_year,r_save_1],axis=0)
    
    ### 完成一个大组的每一年计算后，保存
    r_save_year.to_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'.csv',index=True)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [41]:
bin2s = ['B_M','OP']

for bin2 in bin2s:
    return_df = pd.read_csv('E:/Stock_Data/factor_portfolio/singlebin_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin2_list = [bin2+str(i+1) for i in range(8)]
    bin1_bin2_save_df = pd.DataFrame(index=[''],columns=bin2_list)
    
    col = 0
    for bin2_ in bin2_list:
        bin1_bin2_ = bin2_
        # y减去无风险收益率
        y_temp = return_df.iloc[:1943,:]
        y = y_temp[bin1_bin2_]
        X = dummy_df.iloc[:1943,:]

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()

        # 用OLS
#             X = sm.add_constant(X)
        est = sm.OLS(y,X)
        result = est.fit()           

        calandar_effect = list(result.params)[3]

        # 测试 - 结果矩阵形式保存
        bin1_bin2_save_df.iloc[0,col] = calandar_effect

        col += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/factor_portfolio/'+ bin2 +'3bin_result.csv')

In [68]:
# 5因子回归 - 得系数，用以下一步回归 + 日历效应检验回归，得负周四效应（周四平均超额收益率）
rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

code_lst = os.listdir('E:/Stock_Data/stock_return_data')
result_save_dct = {1:0,2:0,3:0,4:0,5:0}
cnt = 0

f_pvalue = 0
Thu_mean_r = []
for code_csv in code_lst:
    y_temp = pd.read_csv('E:/Stock_Data/stock_return_data/'+code_csv,index_col = 0)
    if len(y_temp)<1943:
        continue
    y = y_temp.iloc[:1943,:]
    X = pd.concat([MKTF,SMB,HML,CMA,RMW],axis=1)
    
    # 用OLS - 1
    X = sm.add_constant(X)
    # 是否使用超额收益率
    y = y - rf

    est = sm.OLS(y,X)
    result = est.fit()
    
    Const = list(result.params)[0]
    MKTF_beta = list(result.params)[1]
    SMB_beta = list(result.params)[2]
    HML_beta = list(result.params)[3]
    CMA_beta = list(result.params)[4]
    RMW_beta = list(result.params)[5]
    
    # 用OLS - 2
    # 是否使用超额收益率
    X = dummy_df.iloc[:1943,]

    est = sm.OLS(y,X)
    result = est.fit()
    
    Thu = list(result.params)[3]
    Thu_Pvalue = list(result.pvalues)[3]
    
    if cnt == 0:
        initial_df = pd.DataFrame(columns=['Code','Thu','Thu_Pvalue','C','MKTF_beta','SMB_beta','HML_beta','CMA_beta','RMW_beta'],index=range(9999))
        initial_df.iloc[cnt,:] = [code_csv[:-4],Thu,Thu_Pvalue,Const,MKTF_beta,SMB_beta,HML_beta,CMA_beta,RMW_beta]
    else:
        initial_df.iloc[cnt,:] = [code_csv[:-4],Thu,Thu_Pvalue,Const,MKTF_beta,SMB_beta,HML_beta,CMA_beta,RMW_beta]
    
    cnt += 1
initial_df = initial_df.dropna()
initial_df.to_csv('E:/Stock_Data/Fama_beta_Thu.csv',index=False)

KeyboardInterrupt: 

In [105]:
initial_df = pd.read_csv('E:/Stock_Data/Fama_beta_Thu.csv')
initial_df['isThu'] = initial_df['Thu_Pvalue'].apply(lambda x: 1 if x<0.01 else 0)

In [162]:
y = initial_df[['Thu']]
y.index=initial_df['Code']
X = initial_df[['C','MKTF_beta','SMB_beta','HML_beta','CMA_beta','RMW_beta']]
X.index=initial_df['Code']
# X = sm.add_constant(X)

est = sm.OLS(y,X)
result = est.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    Thu   R-squared:                       0.855
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     1892.
Date:                Thu, 11 Apr 2019   Prob (F-statistic):               0.00
Time:                        01:52:47   Log-Likelihood:                 10593.
No. Observations:                1934   AIC:                        -2.117e+04
Df Residuals:                    1928   BIC:                        -2.114e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
C              1.0171      0.076     13.372      0.0

In [348]:
# 5因子回归

rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

bin1 = 'Size'
bin2s = ['B_M','OP','Inv']
# bin2s = ['B_M','OP','Inv','pe_ratio','industry']
for bin2 in bin2s:
    file_name = 'E:/Stock_Data/'+bin1+'_'+bin2+'.csv'
    return_df = pd.read_csv('E:/Stock_Data/'+bin1+'_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin1_list = [bin1+str(i+1) for i in range(5)]
    
    bin2_list = [bin2+str(i+1) for i in range(5)]
    bin1_bin2_save_df = pd.DataFrame(index=bin1_list,columns=bin2_list)
    
    row = 0
    for bin1_ in bin1_list:
        col = 0
        for bin2_ in bin2_list:
            bin1_bin2_ = bin1_+bin2_
            # y减去无风险收益率
            y_temp = return_df.iloc[:1943,:]
            y_temp_0 = y_temp[[bin1_bin2_]]
            y_temp_0.columns=['return']
            y = y_temp_0 - rf

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()
            
            # 用OLS
            X = pd.concat([MKTF,SMB,HML,CMA,RMW],axis=1)
            X = sm.add_constant(X)
            est = sm.OLS(y,X)
            result = est.fit()           
            
            save1 = result.rsquared_adj
            save2 = list(result.pvalues)[0]

            # 测试 - 结果矩阵形式保存
            bin1_bin2_save_df.iloc[row,col] = 'adjR2%.2f a_pvalue%.2f'%(save1*100,save2)
            
            col += 1
        row += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/calander_effect_adj/'+bin1+'_'+bin2+'.csv')

In [347]:
# 5因子回归修正

# 引入日历效应哑变量 - 星期
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['weekday']
trade_day_df['weekday'] = trade_day_df['weekday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").weekday()+1)

dummy_df = pd.get_dummies(trade_day_df['weekday'])
dummy_df = dummy_df[[4]]
dummy_df.index = sample_df.index
dummy_df = dummy_df.iloc[:1943,:]

rf = pd.read_csv('E:/Stock_Data/rf.csv',index_col=0)

bin1 = 'Size'
bin2s = ['B_M','OP','Inv']
# bin2s = ['B_M','OP','Inv','pe_ratio','industry']
for bin2 in bin2s:
    file_name = 'E:/Stock_Data/'+bin1+'_'+bin2+'.csv'
    return_df = pd.read_csv('E:/Stock_Data/'+bin1+'_'+bin2+'.csv',index_col=0)
    return_df = return_df.dropna(how="any",axis=0)
    bin1_list = [bin1+str(i+1) for i in range(5)]
    
    bin2_list = [bin2+str(i+1) for i in range(5)]
    bin1_bin2_save_df = pd.DataFrame(index=bin1_list,columns=bin2_list)
    
    row = 0
    for bin1_ in bin1_list:
        col = 0
        for bin2_ in bin2_list:
            bin1_bin2_ = bin1_+bin2_
            # y减去无风险收益率
            y_temp = return_df.iloc[:1943,:]
            y_temp_0 = y_temp[[bin1_bin2_]]
            y_temp_0.columns=['return']
            y = y_temp_0 - rf

#             # 用Garch
#             reg = arch_model(y, x=X, mean='HARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1.0, dist='ged', hold_back=None)
#             result = reg.fit()
            
            # 用OLS
            X = pd.concat([MKTF,SMB,HML,CMA,RMW,dummy_df],axis=1)
            X = sm.add_constant(X)
            est = sm.OLS(y,X)
            result = est.fit()           
            
            save1 = result.rsquared_adj
            save2 = list(result.pvalues)[0]
            save3 = list(result.params)[6]
            save4 = list(result.pvalues)[6]

            # 测试 - 结果矩阵形式保存
            bin1_bin2_save_df.iloc[row,col] = 'adjR2%.2f a_pvalue%.2f Thu%.2f Thu_pvalue%.2f' % (save1*100,save2,save3*100,save4)
            
            col += 1
        row += 1
    # 保存
    bin1_bin2_save_df.to_csv('E:/Stock_Data/calander_effect_adj/'+bin1+'_'+bin2+'adj.csv')

In [341]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     978.6
Date:                Tue, 02 Apr 2019   Prob (F-statistic):               0.00
Time:                        00:00:22   Log-Likelihood:                 6556.3
No. Observations:                1943   AIC:                        -1.310e+04
Df Residuals:                    1936   BIC:                        -1.306e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.806e-05      0.000     -0.228      0.8

In [344]:
result.pvalues

const    8.198361e-01
MKTF     0.000000e+00
SMB      1.309001e-56
HML      1.095246e-45
CMA      8.225096e-02
RMW      7.022682e-01
4        9.014270e-02
dtype: float64